# Setup

In [3]:
%pip install OpenAI

Note: you may need to restart the kernel to use updated packages.


In [4]:

from IPython.display import display, Markdown
import os
import ipywidgets as widgets

### Pretty printing help

In [5]:
import json

def show_json(obj):
    """
    Display the JSON representation of the given object.

    Parameters:
    obj (object): The object to display as JSON.

    Returns:
    None
    """
    display(json.loads(obj.model_dump_json()))

### Load environment variables

In [6]:
from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

## Choose the Example

The following will let the user switch to a different example to run through the pipeline.

In [7]:
def set_destinations(b):
  # This function will be called when the button is clicked
  global choice, job_description_file, company_file, destination_directory, html_destination, pdf_destination, markdown_destination, word_destination
  choice = choice_widget.value
  print("You selected: " + choice)
  # source files
  job_description_file = "examples/" + choice + "/job-description.md"
  company_file = "examples/" + choice + "/about-company.md"

  # destination files
  destination_directory = "examples/" + choice + "/"
  html_destination = destination_directory + 'playbook.html'
  pdf_destination = destination_directory + 'playbook.pdf'
  markdown_destination = destination_directory + 'playbook.md'
  word_destination = destination_directory + 'playbook.docx'


# Which example to run
choice_widget = widgets.Dropdown(
  options=['medical-assistant', 'infra-engineer', 'customer-success-manager', 'real-estate-litigator','growth-manager'],
  value='medical-assistant',
  description='Choose an example:'
)
choice = choice_widget.value

# trigger the button to run the function
set_destinations(choice_widget)

button = widgets.Button(description="Confirm")


button.on_click(set_destinations)

display(choice_widget, button)



You selected: medical-assistant


Dropdown(description='Choose an example:', options=('medical-assistant', 'infra-engineer', 'customer-success-m…

Button(description='Confirm', style=ButtonStyle())

You selected: growth-manager
You selected: infra-engineer


### Job Description

Read the job description into a variable.

In [100]:
job_description_file = "examples/" + choice + "/job-description.md"
with open(job_description_file, "r") as file:
    job_description = file.read()

print(job_description)

## Job Description

**Y Combinator - Infrastructure Software Engineer**

Y Combinator is seeking an Infrastructure Software Engineer who will help manage and evolve the shared infrastructure that our software runs on. We have a team of approximately 15 Product Engineers who develop YC’s software products. Those products are developed in Ruby on Rails and run in AWS. Our team’s job is to provide a platform so our Product Engineers don’t have to think (as much) about infrastructure.

The ideal candidate will have experience debugging and developing a very broad stack. They’re comfortable talking about HTTP caching, AWS security groups, Rails monkey patching, n+1 queries in Active Record, CSRF protection, and React components.

Enjoy the many perks of working for a successful company – competitive salary and excellent benefits including fully-paid health care benefits and unlimited vacation – while working on a small, fast-moving and high-impact team.

In your first month, you might:

- M

### Company Description

Read the company description into a variable.

In [101]:

company_file = "examples/" + choice + "/about-company.md"

with open(company_file, "r") as file:
    company_description = file.read()

print(company_description)

## Y Combinator Mission, Vision, and Values

### Mission

Y Combinator's mission is to increase the number of startups by assisting founders in their creation. The organization aims to get startups through their initial phase, often helping them build something impressive enough to raise significant amounts of money on a larger scale.

### Vision

Y Combinator envisions a world where startups are the norm rather than the exception. They believe that startups are, on balance, a good thing, as their founders and early employees can be much more productive than they would be working for an established company. Y Combinator's vision is to apply mass production techniques to startup funding, making the process more efficient and beneficial for founders.

### Values

Y Combinator values integrity, respect, and accountability. They believe these traits are critical for founders to possess. The organization is built on a network of trust, with its strength and value hinging on the trustworthin

# Develop top-level list of competencies

## Setup OpenAI

In [102]:
from openai import OpenAI
client = OpenAI()
llm = "gpt-4-1106-preview"

## Get Draft List of Competencies

In [128]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. 
Your task is to do a job analysis and identify the knowledge, skills, abilities, and other characteristics (KSAOs) as competencies for a hiring team to assess in a candidate. 
Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the requirements in the job desccription. 
You should also include competencies or word competencies that are aligned with the company values and if a competency is aligned with a company value, you should say so.
Refer to the actual company name rather than 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the key competencies for the job description as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "job_level": "Google Job Level (1-10)",
  "thinking": "Your thoughts about how to tackle this problem",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)


output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)
role = competency_data["role"]
company = competency_data["company"]

{
  "role": "Infrastructure Software Engineer",
  "company": "Y Combinator",
  "job_level": "7",
  "thinking": "This job role requires a blend of technical expertise, especially in AWS, Docker, and web application development, alongside soft skills such as problem-solving and communications. Considering Y Combinator's focus on startups and an efficient, trust-based environment, it's important the candidate demonstrates ability to act swiftly, innovate, and uphold integrity and respect. Y Combinator's culture of enabling rapid startup growth means the candidate should be adept at working in fast-paced, change-prone settings, while maintaining security and performance.",
  "competencies": [
    {
      "name": "Cloud Infrastructure Expertise",
      "description": "Proven ability to develop, deploy, and manage applications within cloud environments, particularly using AWS services such as ECS, RDS, IAM, and CloudWatch. This aligns with Y Combinator's value of applying efficient 'mass pro

## Review and improve the list of competencies

In [129]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to do a job analysis and review the key competencies 
for a hiring team to look for in a candidate. The competencies should recover the knowledge, skills, abilities and other characteristics (KSAOs)
required for the job. Evaluate the provided list of competencies and improve them if necessary.
You will examine the provided competency framework with an eye towards comprehensiveness, relevance, and alignment with industry best practices. 
Your evaluation will focus on ensuring that the framework effectively covers all necessary areas for the {role} at {company} and 
incorporates the company's values wherever practical.
Junior or low-level jobs should have fewer competencies (e.g., 6) and more senior jobs should have more (e.g., 10).
Do not include acronyms or terminology that would not be understood by an industry professional not from the company.
Refer to the actual company name wherever possible rather than use a generic word such as 'organization' or 'company'.
Do NOT ever use the word 'technological' instead use 'technology' or 'technical'.

If you think the list of competencies is good, you can leave it as is.
If you think the list of competencies is bad, you can delete it and write a new one.
If you think the list of competencies is good but needs improvement, you can edit it.
If you think the list of competencies is good but needs more competencies, you can add more.
If you think the list of competencies is good but needs less competencies, you can delete some.
If you think the list of competencies is good but needs to be re-ordered, you can re-order them.
If you think the list of competencies is good but needs to be re-worded, you can re-word them.
If a description is aligned with a company value, you should say so.

The job description is as follows:
{job_description}

The current list of competencies is as follows:
{output}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the improved key competency list as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
      "improvements": "Improvements made (if any)"
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
      "improvements": "Improvements made (if any)"
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
      "improvements": "Improvements made (if any)"
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)


{
  "role": "Infrastructure Software Engineer",
  "company": "Y Combinator",
  "competencies": [
    {
      "name": "Cloud Infrastructure Expertise",
      "description": "Demonstrated capability to build, deploy, and manage applications within cloud environments, with a strong emphasis on AWS services such as ECS, RDS, IAM, and CloudWatch. This competency reflects Y Combinator's pursuit of efficiency and scalability through technology.",
      "improvements": "Clarified the description to emphasize direct experience with specific AWS services."
    },
    {
      "name": "Web Application Development",
      "description": "Proficiency in developing MVC web applications. Familiarity with Y Combinator's tech stack (Ruby on Rails, Postgres, React) is valuable, though not mandatory. Includes the ability to quickly learn and adapt to new frameworks, reflecting the company's value of agile and responsive action.",
      "improvements": "Expanded the description to highlight flexibility and

## Create Markdown for Competency Framework Introduction

In [117]:
markdown_competencies_intro = "\n"
counter = 1
for item in competency_data["competencies"]:
    markdown_competencies_intro += f"""
{counter}. **{item["name"]}**: {item["description"]}

"""
    counter += 1

markdown_competencies_intro = f"""

## Competency Framework

The following competencies have been identified for the {role} role at {company}:

{markdown_competencies_intro}

"""

display(Markdown(markdown_competencies_intro))



## Competency Framework

The following competencies have been identified for the Infrastructure Software Engineer role at Y Combinator:



1. **Technical Proficiency in Web Application Development**: Expertise in the development, debugging, and optimization of MVC web applications, with adeptness in technologies like Ruby on Rails and experience with Postgres.


2. **AWS Platform Mastery**: Comprehensive knowledge of AWS services, especially ECS, RDS, IAM, and CloudWatch, with a strong track record of deploying and managing applications within AWS.


3. **Infrastructure as Code Expertise**: Skilled in using Infrastructure as Code techniques, specifically Terraform, to systematically manage complex infrastructure with code.


4. **Security and Compliance Acumen**: Solid understanding of application and data security principles, ensuring robust compliance and protection, reflecting Y Combinator's value of integrity and trust.


5. **Performance Tuning and Optimization**: Capability to pinpoint and mitigate performance issues in web applications, improving overall efficiency and user experience.


6. **Collaboration and Team Problem-Solving**: Strong communication skills with a team-oriented mindset, able to engage with product engineers and external stakeholders, embodying Y Combinator's values of respect and accountability.


7. **Adaptability and Continuous Learning**: Proven ability to rapidly assimilate new technology stacks and methodologies, demonstrating flexibility and a growth mindset crucial in a fast-paced startup ecosystem.


8. **Proactive Problem-Solving**: Demonstrated initiative in identifying and addressing engineering challenges, often preemptively, which is in line with Y Combinator's proactive and founder-supportive ethos.





## Evaluation scorecard markdown

In [118]:
markdown_evaluation_scorecard = f"""

## Candidate Evaluation

Complete the following scorecard for the {role} role at {company}. Refer to the evaluation guides for 
each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| --- | --- | --- |
"""

spacer = " "

counter = 1
for item in competency_data["competencies"]:
    markdown_evaluation_scorecard += f"""| {counter}. {item["name"]} | {spacer} | {spacer} |
"""
    counter += 1

markdown_evaluation_scorecard += f"""| &nbsp; | | |
| **Total** | {spacer} | {spacer} |

### Other Interviewer Comments:

"""

display(Markdown(markdown_evaluation_scorecard))



## Candidate Evaluation

Complete the following scorecard for the Infrastructure Software Engineer role at Y Combinator. Refer to the evaluation guides for 
each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| --- | --- | --- |
| 1. Technical Proficiency in Web Application Development |   |   |
| 2. AWS Platform Mastery |   |   |
| 3. Infrastructure as Code Expertise |   |   |
| 4. Security and Compliance Acumen |   |   |
| 5. Performance Tuning and Optimization |   |   |
| 6. Collaboration and Team Problem-Solving |   |   |
| 7. Adaptability and Continuous Learning |   |   |
| 8. Proactive Problem-Solving |   |   |
| &nbsp; | | |
| **Total** |   |   |

### Other Interviewer Comments:



# Detailed Competencies

## Function for analyzing a competency in depth

In [130]:
def analyze_competency(competency, competency_description):
    prompt = f"""
You are an Interviewing and Assessment Expert. Your task is to do an analysis of the competency "{competency}" for a job to 
give to a hiring team. 
Do not use the word 'technological' instead use 'technology' or 'technical'.

You should generate a great job-relevant interview question to assess the knowledge, skills, abilities and other characteristics relevant to "{competency}". 
It should be the BEST question you could ask. 
The question should include 1-4 short follow-up questions embedded in the same paragraph. 
Also produce what to look for in the answer to the lead question when evaluating the candidate's answer.

You will then generate a scenario for "{competency}" to ask the candidate to respond to. The scenario should 50 to 75 words.

You will then generate 3 subcompetencies for the "{competency}" competency. Each subcompetency should have a name and a job-relevant interview
question to ask the candidate. Focus on aspects the knowledge, skills, abilities and other characteristics required for this competency
that are not already covered by the lead question.

You will then generate a set of VERY concise evaluation rubrics of less than 18 words to use when evaluating the candidate's answer 
to the lead question and the scenario.
Assume we are using a 5 point scale where 1 is the lowest rating and 5 is the highest rating.
Assume we know we are rating the candidate and don't refer to the candidate.

The competency description is currently as follows. Expand this description in your response if you think it is necessary to vividly describe the ideal candidate
but keep it to about 50 words:
{competency_description}

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the competency analysis list as only a JSON object with no other commentary. Use the following format:

{{"competency": [{{
  "name": "{competency}",
  "description": "{competency_description}",
  "lead_question": "Lead Interview Question",
  "lead_look_for": "What to look for and evaluate in the answer to the lead question. Address the interviewer.",
  "scenario": "A scenario to ask the candidate to respond to",
  "scenario_look_for": "What to look for or evaluate in the answer to the scenario. Address the interviewer.",
  "subcompetencies": [
    {{
      "name": "Subcompetency 1",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 2",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 3",
      "question": "Interview question for Subcompetency"
    }}
  ],
  "rating_1": "What a level 1 rating for this competency looks like. Be very concise.",
  "rating_2: "What a level 2 rating for this competency looks like. Be very concise.",
  "rating_3": "What a level 3 rating for this competency looks like. Be very concise.",
  "rating_4": "What a level 4 rating for this competency looks like. Be very concise.",
  "rating_5": "What a level 5 rating for this competency looks like. Be very concise."
}}"""

    stream = client.chat.completions.create(
        model=llm,
        max_tokens=4096,
        response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    output = ""

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += chunk.choices[0].delta.content

    return json.loads(output)


## Generate detailed competencies

This can take 3-5 minutes to generate.

In [131]:
# intialize the competency details object
competency_details = {"competencies":[]}

# set this to max number of competencies to generate. Use a lower number (e.g., 2) during testing as each competency takes 30+ seconds to generate
max_competencies = 11

# loop through the competencies and generate the competency details
for i, competency in enumerate(competency_data["competencies"]):
    if i == max_competencies:
        break
    competency_deep_dive = analyze_competency(competency["name"], competency["description"])
    competency_details["competencies"].append(competency_deep_dive)

{
  "competency": [{
    "name": "Cloud Infrastructure Expertise",
    "description": "Capability to architect, deploy, and maintain applications within cloud environments, focusing on AWS services like ECS, RDS, IAM, and CloudWatch to drive efficiency and scalability.",
    "lead_question": "Can you walk us through a complex AWS infrastructure you've designed, detailing how you incorporated various AWS services to achieve both scalability and security, and can you explain any trade-offs you had to make in the process, such as cost versus performance, simplicity versus flexibility, or security versus usability?",
    "lead_look_for": "Look for depth in AWS knowledge, examples of critical decision-making regarding trade-offs, understanding scalability, security implications, and cost optimization.",
    "scenario": "Imagine you're tasked with migrating a large, monolithic application to AWS. The goals are to improve scalability, reduce costs, and transition to microservices. Describe yo

## Create Interview Agenda

In [132]:
def generate_interview_agenda(role, company, job_description, company_description, competency_data):    
    prompt = f"""
You are an Interviewing and Assessment Expert. 
Your task is to do an analysis of the job description, the company description, and the competency framework to design an 
agenda for an interview team to use when interviewing a candidate for the {role} role at {company}.
Do not include interview questions, they will be generated later.
Do not include a description of the competencies, just include their name. Details about the competencies will be generated later.
Return the content as markdown text with absolutely no delimiters or commentary.

The agenda should have the following outline. Generate the appropriate text under each heading:
## Interview Agenda
### Welcome and Introduction
### Company Overview
### Job Overview
### Competency Deep Dive
### Candidate Vision and Strategy
### Q&A and Wrap Up

The job description is as follows:
{job_description}

The company description is as follows:
{company_description}

The competency framework is as follows:
{competency_data}
"""

    stream = client.chat.completions.create(
        model=llm,
        max_tokens=4096,
        # response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    output = ""

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += chunk.choices[0].delta.content

    return output

interview_agenda = generate_interview_agenda(role, company, job_description, company_description, competency_data)

## Interview Agenda
### Welcome and Introduction
- Greeting and an icebreaker to set a congenial tone
- Introduce the interview team and their roles
- Outline the structure and timing of the interview

### Company Overview
- Convey Y Combinator's mission to multiply the number of startups
- Share the vision of startups becoming the norm
- Discuss Y Combinator's values: integrity, respect, accountability, efficiency, productivity, and founder-focus

### Job Overview
- Describe the role of Infrastructure Software Engineer at Y Combinator
- Highlight key responsibilities such as managing deployment platforms, ensuring application and data security, and improving developers' tools
- Outline the expected outcomes in the first month on the job
- Location, compensation, and benefits summary

### Competency Deep Dive
- Cloud Infrastructure Expertise
- Web Application Development 
- Containerization Proficiency
- Infrastructure as Code (IaC) Practices
- Performance Debugging
- Security Consciou

## Competency detail sections markdown

In [133]:
counter = 1
markdown_competencies = "\n"

for competency_group in competency_details["competencies"]:
  for competency_item in competency_group["competency"]:

    markdown_competencies += "## Competency " + str(counter) + ". **" 
    markdown_competencies += competency_item["name"] + "**\n\n"

    markdown_competencies += competency_item["description"] + "\n\n"

    markdown_competencies += "### Lead Question\n\n"
    markdown_competencies += "**&ldquo;" + competency_item["lead_question"] + "&rdquo;**\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "_" + competency_item["lead_look_for"] + "_\n\n"

    markdown_competencies += "\n### Situational Scenario\n\n"
    markdown_competencies += "**&ldquo;" + competency_item["scenario"] + "&rdquo;**\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "&nbsp;\n\n"
    markdown_competencies += "_" + competency_item["scenario_look_for"] + "_\n\n"

    markdown_competencies += "### Diving Deeper\n\n"
    for subcompetency in competency_item["subcompetencies"]:
      markdown_competencies += "**" + subcompetency["name"] + "**: \"" + subcompetency["question"] + "\"\n\n"
      markdown_competencies += "&nbsp;\n\n"

    markdown_competencies += "\n### Evaluation\n\n"
    markdown_competencies += "| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |\n"
    markdown_competencies += "| -------- | -------- | -------- | -------- | -------- |\n"
    markdown_competencies += "|" + competency_item["rating_1"] + "|" + competency_item["rating_2"] + "|" + competency_item["rating_3"] + "|" + competency_item["rating_4"] + "|" + competency_item["rating_5"] + "|\n\n"
    
    # markdown_competencies += "\n### Notes\n\n&nbsp;\n\n"

    counter += 1




# Final markdown

In [134]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

January 12, 2024 2:01 PM


In [135]:
# Build final markdown

final_markdown = f"""--- 
title: Interview Guide for {role}
subtitle: {company}
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: {formatted_now}
--- 

# Interview Guide for {role}

## About this Guide

Welcome to this strategic guide to streamline and enhance the interview process for the position of {role}. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
Through structured interviewing, our aim is to identify a candidate who not only possesses the necessary technical skills and experience but also aligns with 
{company}'s values and culture. 

The playbook outlines a clear structure for the interview process, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and {company}.

The guide includes both behavioral and situational interview questions. We recommend dividing up competencies and questions between two or three interviewers
and using a combination of both types to assess a candidate's suitability for the role. 

- **Behavioral** questions are based on the premise that past behavior predicts future performance. These questions prompt candidates to recount specific 
past experiences, focusing on their actions and outcomes. They are effective for roles where past experience is a strong indicator of success, 
especially in skills like problem-solving and leadership. For example, asking "Describe a time when you worked under a tight deadline" assesses 
time management and pressure handling.

- **Situational** questions are hypothetical and designed to assess how candidates might handle future situations. They test judgment and problem-solving 
abilities, asking candidates to respond to imaginary scenarios. These questions are useful for evaluating less experienced candidates or roles 
requiring adaptability. An example is, "How would you handle a project falling behind schedule?" which gauges analytical thinking and adaptability.

Both behavioral and situational questions are valuable in interviews 
but serve different purposes. Behavioral questions are best for roles where past experience predicts success, while situational questions 
suit roles requiring future problem-solving and adaptability. A combination of both types offers a comprehensive assessment of a candidate's capabilities.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) using GPT-4 and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).

{company_description}

{job_description}

{interview_agenda}

{markdown_competencies_intro}

{markdown_competencies}

{markdown_evaluation_scorecard}
"""

display(Markdown(final_markdown))

# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(final_markdown)

--- 
title: Interview Guide for Infrastructure Software Engineer
subtitle: Y Combinator
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 12, 2024 2:01 PM
--- 

# Interview Guide for Infrastructure Software Engineer

## About this Guide

Welcome to this strategic guide to streamline and enhance the interview process for the position of Infrastructure Software Engineer. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
Through structured interviewing, our aim is to identify a candidate who not only possesses the necessary technical skills and experience but also aligns with 
Y Combinator's values and culture. 

The playbook outlines a clear structure for the interview process, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and Y Combinator.

The guide includes both behavioral and situational interview questions. We recommend dividing up competencies and questions between two or three interviewers
and using a combination of both types to assess a candidate's suitability for the role. 

- **Behavioral** questions are based on the premise that past behavior predicts future performance. These questions prompt candidates to recount specific 
past experiences, focusing on their actions and outcomes. They are effective for roles where past experience is a strong indicator of success, 
especially in skills like problem-solving and leadership. For example, asking "Describe a time when you worked under a tight deadline" assesses 
time management and pressure handling.

- **Situational** questions are hypothetical and designed to assess how candidates might handle future situations. They test judgment and problem-solving 
abilities, asking candidates to respond to imaginary scenarios. These questions are useful for evaluating less experienced candidates or roles 
requiring adaptability. An example is, "How would you handle a project falling behind schedule?" which gauges analytical thinking and adaptability.

Both behavioral and situational questions are valuable in interviews 
but serve different purposes. Behavioral questions are best for roles where past experience predicts success, while situational questions 
suit roles requiring future problem-solving and adaptability. A combination of both types offers a comprehensive assessment of a candidate's capabilities.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) using GPT-4 and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).

## Y Combinator Mission, Vision, and Values

### Mission

Y Combinator's mission is to increase the number of startups by assisting founders in their creation. The organization aims to get startups through their initial phase, often helping them build something impressive enough to raise significant amounts of money on a larger scale.

### Vision

Y Combinator envisions a world where startups are the norm rather than the exception. They believe that startups are, on balance, a good thing, as their founders and early employees can be much more productive than they would be working for an established company. Y Combinator's vision is to apply mass production techniques to startup funding, making the process more efficient and beneficial for founders.

### Values

Y Combinator values integrity, respect, and accountability. They believe these traits are critical for founders to possess. The organization is built on a network of trust, with its strength and value hinging on the trustworthiness of its members. Unethical behavior is not tolerated, as it puts the reputation of the entire community at risk. Y Combinator also values the idea of putting the founders' interests first, even before their own. This approach, while seemingly counterintuitive in a for-profit business, has proven to be more scalable and beneficial in the long run.

Y Combinator also values evidence of demand for a startup's solution and the strength of the founding team. They believe in the importance of launching and testing ideas quickly, rather than overplanning or overthinking. This approach is based on the understanding that the market and user needs can change rapidly, and startups need to be agile and responsive to these changes.

In summary, Y Combinator's mission, vision, and values revolve around fostering a supportive and ethical environment for startups, promoting efficiency and productivity, and encouraging quick action and responsiveness to market demands.


## Job Description

**Y Combinator - Infrastructure Software Engineer**

Y Combinator is seeking an Infrastructure Software Engineer who will help manage and evolve the shared infrastructure that our software runs on. We have a team of approximately 15 Product Engineers who develop YC’s software products. Those products are developed in Ruby on Rails and run in AWS. Our team’s job is to provide a platform so our Product Engineers don’t have to think (as much) about infrastructure.

The ideal candidate will have experience debugging and developing a very broad stack. They’re comfortable talking about HTTP caching, AWS security groups, Rails monkey patching, n+1 queries in Active Record, CSRF protection, and React components.

Enjoy the many perks of working for a successful company – competitive salary and excellent benefits including fully-paid health care benefits and unlimited vacation – while working on a small, fast-moving and high-impact team.

In your first month, you might:

- Migrate a Rails application from Capistrano and EC2 to Docker and ECS
- Upgrade one of our applications to a newer version of Postgres or Rails
- Move more of our AWS infrastructure into Terraform
- Simplify our use of Webpack and Javascript for both developer productivity and faster deployments
- Create shared infrastructure to help manage email subscriptions and unsubscribes across 5 different product teams
- Triage and respond to bug bounty reports
- Work with outside security auditors to run a penetration test against our applications

**Key Responsibilities:**

- Provide a stable modern platform for our applications to be deployed to
- Keep our applications and data secure
- Improve our developer experience and tools so the YC Software Team can ship product updates quickly
- Help debug performance issues in our web applications
- Ship Ruby on Rails and TypeScript code that is shared among multiple YC applications

**Skills:**

- You know how to develop and debug MVC web applications. We use Ruby on Rails, Postgres, and React, but don’t worry if you haven’t used those specific technologies before
- You know how to deploy and run applications with AWS and Docker
- You use Terraform to make infrastructure changes

**Experience:**

- You have at least 5 years of experience developing and deploying web applications
- You are deeply familiar with the AWS platform specifically ECS, RDS, IAM, and CloudWatch

**Location:** YC is headquartered in the SF Bay Area with employees working from home or from our offices in San Francisco and Mountain View, CA. Candidates must live in the SF Bay Area or be willing to relocate.

**Compensation:** $130,000 to $270,000 + bonus (depending on skills and experience).

**Benefits:** Our full benefits package includes medical, vision, and dental plans, infertility benefit, STD/LTD, life insurance, commuter benefits, flexible spending account, health savings account, 401(k) + 4% matching, generous parental leave, paid holidays and flexible paid time off policy.

**Work Authorization:** This position does not support work authorization/visa sponsorship.

Y Combinator considers qualified applicants with criminal histories, consistent with applicable federal, state, and local law including San Francisco’s Fair Chance Ordinance. Y Combinator is committed to protecting the privacy of the personal information of job applicants and complying with the California Consumer Privacy Act. The privacy policy of Ashby, Inc., the hiring platform used by Y Combinator, governs the collection of such data.

## Interview Agenda
### Welcome and Introduction
- Greeting and an icebreaker to set a congenial tone
- Introduce the interview team and their roles
- Outline the structure and timing of the interview

### Company Overview
- Convey Y Combinator's mission to multiply the number of startups
- Share the vision of startups becoming the norm
- Discuss Y Combinator's values: integrity, respect, accountability, efficiency, productivity, and founder-focus

### Job Overview
- Describe the role of Infrastructure Software Engineer at Y Combinator
- Highlight key responsibilities such as managing deployment platforms, ensuring application and data security, and improving developers' tools
- Outline the expected outcomes in the first month on the job
- Location, compensation, and benefits summary

### Competency Deep Dive
- Cloud Infrastructure Expertise
- Web Application Development 
- Containerization Proficiency
- Infrastructure as Code (IaC) Practices
- Performance Debugging
- Security Consciousness
- Developer Tooling Creation
- Effective Communication
- Adaptability and Flexibility
- Cross-functional Collaboration
- Strategic Thinking and Planning

### Candidate Vision and Strategy
- Discussion about the candidate's future vision for the role
- Explore how the candidate would approach strategic planning for infrastructure development

### Q&A and Wrap Up
- Invite the candidate to ask any questions they have about the role, team, or company
- Provide information on the next steps in the selection process
- Thank the candidate for their time and participation



## Competency Framework

The following competencies have been identified for the Infrastructure Software Engineer role at Y Combinator:



1. **Technical Proficiency in Web Application Development**: Expertise in the development, debugging, and optimization of MVC web applications, with adeptness in technologies like Ruby on Rails and experience with Postgres.


2. **AWS Platform Mastery**: Comprehensive knowledge of AWS services, especially ECS, RDS, IAM, and CloudWatch, with a strong track record of deploying and managing applications within AWS.


3. **Infrastructure as Code Expertise**: Skilled in using Infrastructure as Code techniques, specifically Terraform, to systematically manage complex infrastructure with code.


4. **Security and Compliance Acumen**: Solid understanding of application and data security principles, ensuring robust compliance and protection, reflecting Y Combinator's value of integrity and trust.


5. **Performance Tuning and Optimization**: Capability to pinpoint and mitigate performance issues in web applications, improving overall efficiency and user experience.


6. **Collaboration and Team Problem-Solving**: Strong communication skills with a team-oriented mindset, able to engage with product engineers and external stakeholders, embodying Y Combinator's values of respect and accountability.


7. **Adaptability and Continuous Learning**: Proven ability to rapidly assimilate new technology stacks and methodologies, demonstrating flexibility and a growth mindset crucial in a fast-paced startup ecosystem.


8. **Proactive Problem-Solving**: Demonstrated initiative in identifying and addressing engineering challenges, often preemptively, which is in line with Y Combinator's proactive and founder-supportive ethos.






## Competency 1. **Cloud Infrastructure Expertise**

Capability to architect, deploy, and maintain applications within cloud environments, focusing on AWS services like ECS, RDS, IAM, and CloudWatch to drive efficiency and scalability.

### Lead Question

**&ldquo;Can you walk us through a complex AWS infrastructure you've designed, detailing how you incorporated various AWS services to achieve both scalability and security, and can you explain any trade-offs you had to make in the process, such as cost versus performance, simplicity versus flexibility, or security versus usability?&rdquo;**

&nbsp;

&nbsp;

_Look for depth in AWS knowledge, examples of critical decision-making regarding trade-offs, understanding scalability, security implications, and cost optimization._


### Situational Scenario

**&ldquo;Imagine you're tasked with migrating a large, monolithic application to AWS. The goals are to improve scalability, reduce costs, and transition to microservices. Describe your approach.&rdquo;**

&nbsp;

&nbsp;

_Expect thoughtful microservice architecture proposal, clear migration plan, considerations for cost-savings and scalability, and awareness of potential challenges._

### Diving Deeper

**Security and Compliance**: "How do you establish and maintain security and compliance within an AWS cloud environment, especially considering legal regulations like GDPR or HIPAA?"

&nbsp;

**Performance Optimization**: "Explain a time when you had to optimize the performance of AWS resources. Which services did you tweak, and what metrics did you use to measure improvements?"

&nbsp;

**Cost Management**: "Describe your experience with managing and reducing costs in an AWS environment. What tools or strategies do you deploy for cost-effective cloud management?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Lacks understanding of AWS, can't discuss design trade-offs.|Basic AWS knowledge, struggles with scalability/security balance.|Competent in AWS, some insight into design decisions and scaling.|Strong in AWS, clear design process, balances trade-offs well.|Expert-level AWS architect, excels in scalable/secure design, mindful of trade-offs.|

## Competency 2. **Web Application Development**

Expertise in crafting MVC web applications with an emphasis on Y Combinator's tech stack, including Ruby on Rails, Postgres, and React. Ability to assimilate new frameworks rapidly, demonstrating flexibility and a proactive approach in a changing environment.

### Lead Question

**&ldquo;Describe a complex web application you have developed. What were the main challenges you encountered and how did you overcome them? Can you discuss any optimizations you implemented and the impact they had on the application's performance?&rdquo;**

&nbsp;

&nbsp;

_Look for detailed explanations of projects, clear understanding of specific challenges, innovative problem-solving methods, and tangible results related to performance improvements._


### Situational Scenario

**&ldquo;Imagine you need to optimize a slow-loading dashboard in a Ruby on Rails application. It pulls data from multiple Postgres tables and renders several React components. How would you approach this task?&rdquo;**

&nbsp;

&nbsp;

_Expect a methodical approach to profiling the application, identifying bottlenecks, and a range of optimization strategies for both backend and frontend._

### Diving Deeper

**Framework Proficiency**: "How would you transition an application from Ruby on Rails to another MVC framework, and what factors would you consider in making this decision?"

&nbsp;

**Database Optimization**: "Can you provide an example of a time when you optimized a database query in Postgres? What steps did you take and what was the outcome?"

&nbsp;

**Responsive Frontend Development**: "How do you ensure that the React components you develop are both performant and responsive across different devices and browsers?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Cannot describe project challenges or optimizations.|Describes basic challenges and standard optimizations with vague outcomes.|Identifies challenges, describes common solutions with some results.|Provides clear examples, innovative solutions, and demonstrates impact on performance.|Expertly details complex challenges, unique solutions, and substantial performance gains.|

## Competency 3. **Containerization Proficiency**

Expertise in deploying applications using Docker and orchestrating with Kubernetes or similar systems. Critical for building scalable, efficient, and modern technological environments at Y Combinator.

### Lead Question

**&ldquo;Can you describe a complex application you have containerized, explaining how you handled persistent storage, networking, and service discovery? What challenges did you face during the containerization process and how did you overcome them?&rdquo;**

&nbsp;

&nbsp;

_Look for depth of experience with Docker, awareness of container networking/storage complexities, problem-solving skills, and understanding of service discovery mechanisms._


### Situational Scenario

**&ldquo;Imagine you're tasked to containerize a legacy monolithic Ruby on Rails application. It frequently writes to a filesystem and has stringent uptime requirements. How would you approach this project?&rdquo;**

&nbsp;

&nbsp;

_Expect strategic thinking around containerizing legacy apps, considering state persistence, minimizing downtime, and practical steps for migration._

### Diving Deeper

**Docker Command Proficiency**: "What Docker commands do you use most frequently, and can you give an example of how you've used a complex command or combination of commands in practice?"

&nbsp;

**Container Orchestration Knowledge**: "How would you explain the difference between Docker Swarm and Kubernetes to someone new to containerization, and why might you choose one over the other?"

&nbsp;

**CI/CD Integration**: "How do you integrate containerization into a CI/CD pipeline, and can you describe a time when this improved the development process?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Limited knowledge, can only describe basic concepts.|Basic Docker use, little orchestration, limited problem-solving shown.|Adequate Docker use, basic orchestration, addresses simple scenarios.|Advanced Docker skills, orchestration proficiency, strong problem-solving.|Expert in Docker, orchestrates like Kubernetes, innovative, solves complex issues.|

## Competency 4. **Infrastructure as Code (IaC) Practices**

Possessing advanced expertise in Infrastructure as Code, especially with Terraform proficiency, this role focusses on utilizing IaC to automate and upkeep infrastructure, thereby boosting development speed and reliability, which is crucial for Y Combinator's operational effectiveness.

### Lead Question

**&ldquo;Describe a complex environment you have automated using Infrastructure as Code with Terraform. How did you manage state files, handle changes in infrastructure without downtime, and ensure security best practices were followed?&rdquo;**

&nbsp;

&nbsp;

_Look for detailed description of environment, clear understanding of Terraform state management, strategies to prevent downtime during updates, and incorporation of security practices into their workflow._


### Situational Scenario

**&ldquo;Imagine we're migrating several applications to AWS, using Terraform for setup. Halfway through, AWS releases new service updates. How would you incorporate these updates into the IaC scripts without disrupting ongoing operations?&rdquo;**

&nbsp;

&nbsp;

_Evaluate how they plan to test and integrate new service features, manage potential risks, ensure minimal disruption, and their thought process on maintaining IaC scripts._

### Diving Deeper

**Version Control in IaC**: "How do you implement version control in Terraform for IaC to manage collaboration and track changes?"

&nbsp;

**IaC Testing and Validation**: "What methods do you use for testing and validating your infrastructure code before and after deployment?"

&nbsp;

**IaC Scalability and Modularity**: "Can you describe a system where you applied modularity and scalability practices in IaC? What challenges did you encounter?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Shows minimal understanding; unable to elaborate on Terraform usage.|Limited detail in application; basic grasp of Terraform without depth.|Adequate Terraform use; some integration and security practices known.|Strong Terraform knowledge; competent state handling and security integration.|Expert level; excels in complex IaC, Terraform mastery with zero downtime solutions.|

## Competency 5. **Performance Debugging**

Expertise in identifying and repairing web application performance issues, understanding n+1 query problems, implementing caching strategies, and resolving performance bottlenecks in a manner that promotes rapid iteration and elevated productivity.

### Lead Question

**&ldquo;Describe a complex performance issue you encountered in a web application and how you went about diagnosing and resolving it. What tools did you use, and how did you verify that the issue was resolved?&rdquo;**

&nbsp;

&nbsp;

_Look for specific examples, clarity of problem-solving process, use of relevant tools/frameworks, and evidence of a results-oriented approach._


### Situational Scenario

**&ldquo;Your Rails application's response time has spiked suddenly. Users are reporting slow page loads consistently. Describe the steps you would take to debug this issue.&rdquo;**

&nbsp;

&nbsp;

_Candidate should outline a structured approach to diagnosis, including log analysis, performance metrics, and hypothesis-driven testing._

### Diving Deeper

**Diagnostic Acumen**: "Can you discuss a time when you successfully identified a performance bottleneck that was not immediately apparent? What was your thought process?"

&nbsp;

**Optimization Knowledge**: "How do you prioritize which performance optimizations to implement in a new or existing web application?"

&nbsp;

**Tool Proficiency**: "What performance profiling tools are you most comfortable using, and can you give an example of how you've used them?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Lacks clarity, no clear process, unfamiliarity with tools.|Some process understanding, little hands-on experience.|Adequate process and tool knowledge, few practical examples.|Clear process demonstrated, proficient in standard tools, relevant examples given.|Deep understanding, strong process and extensive tool use, insightful practical examples.|

## Competency 6. **Security Consciousness**

Demonstrating a persistent awareness of the importance of security within technology infrastructure, proactively implementing secure coding practices, comprehending CSRF, XSS and other common web vulnerabilities and how to defend against them, all within the framework of maintaining the integrity and accountability values of Y Combinator.

### Lead Question

**&ldquo;Tell me about a time you identified and addressed a security vulnerability within an application or infrastructure you were responsible for. How did you approach the situation, and what measures did you implement to prevent future occurrences?&rdquo;**

&nbsp;

&nbsp;

_Look for detail in identifying the vulnerability, understanding its potential impact, practices the candidate used to mitigate it, and their approach to prevent similar issues in the future._


### Situational Scenario

**&ldquo;Imagine you've deployed a new feature which inadvertently introduced a CSRF vulnerability. A hacker could potentially exploit this to control user accounts. How would you resolve this issue?&rdquo;**

&nbsp;

&nbsp;

_Expect a clear resolution plan, understanding the urgency, depth in CSRF knowledge, and steps to secure the application moving forward._

### Diving Deeper

**Vulnerability Assessment**: "How do you stay up-to-date with new security threats, and what is your process for assessing vulnerabilities in a codebase?"

&nbsp;

**Security Best Practices**: "Can you describe how you ensure secure software development lifecycle in your projects?"

&nbsp;

**Incident Response**: "Describe the steps you would take if you discovered a live security breach."

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Minimal awareness, no proactive stance.|Basic awareness, some knowledge gaps.|Adequate knowledge, moderate proactivity.|Strong knowledge and problem-solving capability.|Expert with extensive proactive measures.|

## Competency 7. **Developer Tooling Creation**

The ability to enhance the developer experience through the creation and maintenance of developer tools, aiding in productivity and rapid deployment. Essential for Y Combinator's Infrastructure Software Engineers, enabling scalable and efficient development practices.

### Lead Question

**&ldquo;Tell me about a time you identified a need for a new developer tool and how you went about designing, implementing, and sharing it with your team. What were the challenges you faced and how did you ensure the tool was adopted?&rdquo;**

&nbsp;

&nbsp;

_Look for demonstrated ability in tool creation, understanding of development challenges, user-centric design, effective communication skills, and methods to encourage adoption._


### Situational Scenario

**&ldquo;Imagine you've noticed our engineers frequently encountering the same n+1 query issue in Active Record. How would you approach creating a tool to help mitigate this recurring problem?&rdquo;**

&nbsp;

&nbsp;

_Expect a thoughtful approach to problem-solving, familiarity with Active Record, a focus on prevention, and a strategic plan for tool integration._

### Diving Deeper

**Development Workflow Optimization**: "Describe the process you use to evaluate which aspects of the development workflow could benefit most from new tools or improvements to existing ones."

&nbsp;

**Tool Usability and Training**: "What strategies do you employ to ensure the tools you develop are user-friendly and widely adopted by the developer team?"

&nbsp;

**Tool Maintenance and Evolution**: "How do you manage the ongoing maintenance and improvement of tools you've developed, and how do you collect feedback for iterative development?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Shows limited insight into tool creation or impact.|Understands basics but lacks comprehensive strategy.|Demonstrates adequate skill, some effective tooling examples.|Provides detailed examples with effective results, good understanding.|Exhibits exceptional skill, strategic vision, and evidence of impactful tools.|

## Competency 8. **Effective Communication**

This role requires superior communication skills for articulating complex technology issues, facilitating tech discussions among team members, and collaborating effectively within a team structure while upholding Y Combinator's values of respect and integrity towards all stakeholders.

### Lead Question

**&ldquo;Tell me about a situation where you had to explain a complex technology problem to a team member or stakeholder with non-technical expertise. How did you ensure they understood it?&rdquo;**

&nbsp;

&nbsp;

_Look for clarity in explanation, use of non-technical language, checking for understanding, and empathy for the listener's technical level._


### Situational Scenario

**&ldquo;You need to explain why a Rails upgrade is critical for security to a non-technical stakeholder who is concerned about the cost and time it will take. How would you approach this conversation?&rdquo;**

&nbsp;

&nbsp;

_Expect tactful explanations, appreciation of cost concerns, clear communication of benefits versus risks, and persuasion skills._

### Diving Deeper

**Technical Literacy and Clarity**: "How do you keep your technical knowledge up to date, and can you share how you would simplify a technology concept that you recently learned?"

&nbsp;

**Active Listening and Empathy**: "Can you describe a time when you had to actively listen to a colleague's concerns about a technical project? How did you address those concerns?"

&nbsp;

**Writing and Documentation**: "Writing clear documentation is critical. Can you discuss your experience with documenting technical processes and how you ensure they are user-friendly?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Avoided technical topics, ignored listener feedback.|Limited clarity, technical terms not explained well.|Clearly explains technology with some listener engagement.|Effectively demystifies technology, seeks and uses feedback.|Masters technical explanations, fully engages listener, and displays empathy.|

## Competency 9. **Adaptability and Flexibility**

Ability to quickly assimilate new technologies and methodologies, which is essential in Y Combinator's dynamic startup environment, reflecting the company's value of responsiveness to change, and effectively navigating frequent pivoting of project focus.

### Lead Question

**&ldquo;Can you describe a situation where you had to adapt to a significant change in technology or direction in a project mid-way, and how did you handle the transition? Did you encounter any challenges, and how did you ensure the new approach was successfully implemented?&rdquo;**

&nbsp;

&nbsp;

_Look for instances of proactive learning, resourcefulness, and resilience. Evaluate the effectiveness of the transition and the ability to maintain performance standards._


### Situational Scenario

**&ldquo;Imagine you're leading the migration of a Rails application to Docker and ECS, and halfway through, there's an urgent need to integrate a new security layer unforeseen due to recent vulnerability reports. How would you proceed?&rdquo;**

&nbsp;

&nbsp;

_Assess for quick re-prioritization skills, problem-solving aptitude, clear communication, and the ability to maintain calm under unexpected changes._

### Diving Deeper

**Proactive Learning**: "How do you stay updated with the latest developments in our tech stack, and can you give an example where this knowledge directly impacted your work?"

&nbsp;

**Resource Management**: "Tell me about a time when resource constraints required you to rethink your approach to a project or task."

&nbsp;

**Solution-Oriented Mindset**: "When faced with an unexpected problem during a project, how do you structure your problem-solving process?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Struggles with change, avoids new methods, lacks examples of adaptability.|Manages minor changes, requires guidance for new methods, some adaptability.|Accepts change, uses new methods with some efficacy, provides basic adaptability examples.|Adapts well, learns new methods quickly, offers solid examples of flexibility.|Excels in changing environments, masters new methods innovatively, demonstrates strategic adaptability.|

## Competency 10. **Cross-functional Collaboration**

This competency involves demonstrating the ability to lead and drive initiatives that necessitate collaboration and problem-solving among diverse functional groups. It's key to foster a culture of cooperation within YC’s ecosystem of startups.

### Lead Question

**&ldquo;Can you tell us about a time when you had to work collaboratively with different teams or departments to achieve a common goal? What was your role in facilitating this collaboration and how did you ensure the involvement and buy-in from all parties? Were there any conflicts, and how were they resolved?&rdquo;**

&nbsp;

&nbsp;

_Look for specific examples highlighting communication skills, leadership in team settings, tactics to gain buy-in, conflict resolution strategies, and the ability to unite diverse groups towards a common goal._


### Situational Scenario

**&ldquo;Imagine you're rolling out a new technology infrastructure that affects multiple departments. Your approach has met resistance from a department that is unfamiliar with the new technology. How would you handle this situation?&rdquo;**

&nbsp;

&nbsp;

_Assess the candidate's strategy for managing resistance, their communication skills, how they educate and motivate team members, and their adaptability to feedback and different departmental perspectives._

### Diving Deeper

**Communication & Persuasion**: "How do you adjust your communication style when explaining technical details to non-technical team members to ensure understanding and support?"

&nbsp;

**Conflict Resolution & Negotiation**: "Describe a time when you had to negotiate or mediate a conflict between team members from different functional areas. What was the outcome?"

&nbsp;

**Planning & Project Management**: "Tell me about a project you managed that required cross-functional input. How did you coordinate the various contributions and keep the project on track?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Handles individual tasks, but struggles with group coordination.|Participates within group settings but lacks proactive leadership.|Adequately facilitates group projects; occasional leadership shown.|Consistently leads cross-functional efforts effectively.|Excels at uniting diverse groups, fosters outstanding collaboration.|

## Competency 11. **Strategic Thinking and Planning**

The capacity to contribute to strategic decisions about the infrastructure in line with broader engineering and company goals, showcasing an ability to balance immediate technical challenges with long-term innovation. This includes considering the implications of technical decisions on business outcomes and the ability to forecast future needs and trends.

### Lead Question

**&ldquo;Can you describe a time when you had to develop a strategic plan for technology infrastructure that included both immediate and long-term considerations? How did you approach trade-offs, and how did you ensure alignment with broader company goals? Please provide specifics on the process and outcome.&rdquo;**

&nbsp;

&nbsp;

_Look for detailed planning process, clear trade-off consideration, alignment with company goals, and tangible results. Evaluate the depth of strategic thinking and long-term vision._


### Situational Scenario

**&ldquo;Imagine our company plans to significantly scale its customer base in the next 12 months. Outline your strategic plan for preparing our infrastructure to handle this growth while maintaining performance and security standards.&rdquo;**

&nbsp;

&nbsp;

_Look for a comprehensive plan that covers scalability, performance, security, and possible contingencies. Evaluate clarity, depth, and proactiveness._

### Diving Deeper

**Innovation Forecasting**: "How do you stay informed about the latest industry trends, and how have you applied this knowledge in past strategic planning?"

&nbsp;

**Business-Technology Alignment**: "Can you give an example of how you aligned a technical project with the broader business objectives? What measures did you take to ensure alignment?"

&nbsp;

**Decision Making and Prioritization**: "Talk about a situation where you had to prioritize certain infrastructure improvements over others. What criteria did you use for prioritization?"

&nbsp;


### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Shows fundamental misunderstanding of strategic planning.|Displays basic awareness but lacks depth in strategic planning.|Develops adequate plans with some strategic foresight.|Exhibits a high level of strategic planning with clear foresight.|Masterful in strategic planning with exceptional foresight and outcomes.|





## Candidate Evaluation

Complete the following scorecard for the Infrastructure Software Engineer role at Y Combinator. Refer to the evaluation guides for 
each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| --- | --- | --- |
| 1. Technical Proficiency in Web Application Development |   |   |
| 2. AWS Platform Mastery |   |   |
| 3. Infrastructure as Code Expertise |   |   |
| 4. Security and Compliance Acumen |   |   |
| 5. Performance Tuning and Optimization |   |   |
| 6. Collaboration and Team Problem-Solving |   |   |
| 7. Adaptability and Continuous Learning |   |   |
| 8. Proactive Problem-Solving |   |   |
| &nbsp; | | |
| **Total** |   |   |

### Other Interviewer Comments:




## Build Word docx

In [136]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [137]:
# Convert from markdown to HTML

!pandoc -s {markdown_destination} -o {html_destination} --css=../../templates/html-styles.css --reference-doc="templates/html-reference.html" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [138]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
HTML(html_destination).write_pdf(pdf_destination, stylesheets=['templates/html-styles.css'], full_fonts=True, hinting=True, presentational_hints=True)